### Gemma IT 7B
#### RAG with LlamaIndex - Nvidia CUDA + WSL (Windows Subsystem for Linux) + Word documents + Local LLM

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation using Windows' WSL and an Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

/home/markwsl/miniconda3/envs/LlamaIndexRAGLinux/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/gemma-7b-it-q8_0.gguf', # 8-bit model
    temperature=0.1,
    max_new_tokens=1024, # Increasing to support longer responses
    context_window=8192, # 8K context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 40}, # 40 was a good amount of layers for the RTX 3090, you may need to decrease yours if you have less VRAM than 24GB
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 21 key-value pairs and 254 tensors from ./Models/gemma-7b-it-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 28
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   5:                  gemma.feed_forward_length

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large", cache_folder=None)

#### 6. Prompt Template

Prompt template:

```
<start_of_turn>user
Question here<end_of_turn>
<start_of_turn>model
```

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256 # Number of tokens in each chunk

#### 8. Index documents

In [6]:
index = VectorStoreIndex.from_documents(documents)

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [7]:
from llama_index.core.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is in the question, adjust it to match what you want the LLM to act like and do.

In [8]:
def RunQuestion(questionText):

    queryQuestion = "You are a technology specialist. Answer questions in a positive, helpful and empathetic way. Answer the following question: " + questionText + ""

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [9]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [10]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me



llama_print_timings:        load time =    5359.76 ms
llama_print_timings:      sample time =     607.49 ms /   196 runs   (    3.10 ms per token,   322.64 tokens per second)
llama_print_timings: prompt eval time =    5579.59 ms /  1005 tokens (    5.55 ms per token,   180.12 tokens per second)
llama_print_timings:        eval time =    4425.53 ms /   195 runs   (   22.70 ms per token,    44.06 tokens per second)
llama_print_timings:       total time =   14953.76 ms /  1200 tokens
Llama.generate: prefix-match hit


1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =    5359.76 ms
llama_print_timings:      sample time =     186.71 ms /    54 runs   (    3.46 ms per token,   289.22 tokens per second)
llama_print_timings: prompt eval time =     350.97 ms /   794 tokens (    0.44 ms per token,  2262.30 tokens per second)
llama_print_timings:        eval time =    1296.01 ms /    53 runs   (   24.45 ms per token,    40.89 tokens per second)
llama_print_timings:       total time =    3123.94 ms /   847 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 3.docx| Source 1:
2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated to designated shelters, the Thundertooth family sprang into action. Lumina worked tirelessly to strengthen the city's energy systems, Echo echoed evacuation orders through the city's speakers, Sapphire offered comfort to those in distress, and Ignis unleashed controlled bursts of flames towards the approaching meteor.



Thundertooth stood at the forefront, using his mighty roar to coordinate and inspire th


llama_print_timings:        load time =    5359.76 ms
llama_print_timings:      sample time =     517.96 ms /   147 runs   (    3.52 ms per token,   283.80 tokens per second)
llama_print_timings: prompt eval time =     336.51 ms /   765 tokens (    0.44 ms per token,  2273.35 tokens per second)
llama_print_timings:        eval time =    3748.88 ms /   146 runs   (   25.68 ms per token,    38.94 tokens per second)
llama_print_timings:       total time =    8070.21 ms /   911 tokens
Llama.generate: prefix-match hit


1/4: |Thundertooth Part 2.docx| Source 1:
Thundertooth's toy factory became a sensation, and its creations were highly sought after. The widgets incorporated cutting-edge holographic displays, levitation technology, and even the ability to change shapes and colors with a mere thought. Children across the city rejoiced as they played with these incredible toys that seemed to bring their wildest fantasies to life.



As the years passed, Thundertooth's life took a heartwarming turn. He met a kind and intelligent dinosaur named Seraphina, and together they started a family. Thundertooth and Seraphina were blessed with four children, each with unique characteristics that mirrored the diversity of their modern world.



Lumina: The eldest of Thundertooth's children, Lumina inherited her mother's intelligence and her father's sense of wonder. With sparkling scales that emitted a soft glow, Lumina had the ability to generate light at will. She became fascinated with technology, often spending


llama_print_timings:        load time =    5359.76 ms
llama_print_timings:      sample time =     334.01 ms /   105 runs   (    3.18 ms per token,   314.36 tokens per second)
llama_print_timings: prompt eval time =     280.31 ms /   723 tokens (    0.39 ms per token,  2579.25 tokens per second)
llama_print_timings:        eval time =    2344.36 ms /   104 runs   (   22.54 ms per token,    44.36 tokens per second)
llama_print_timings:       total time =    5101.11 ms /   827 tokens


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth nodded, understanding the gravity of the situation. He gathered Lumina, Echo, Sapphire, and Ignis, explaining the urgency and the role each of them would play in the impending crisis.



1. **Lumina**: Utilizing her deep understanding of technology, Lumina would enhance the city's energy systems to generate a powerful force field, providing a protective barrier against the meteor's impact.



2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated t

#### 13. Output responses

In [11]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

The story is about Thundertooth's journey through time to Mayor Grace's city in which he found sustenance while navigating through futuristic marketplaces with innovative food labs that ultimately led to his well-being in this technological jungle of flying cars  [3]. The story highlights themes of unity between past  and future through compassion  and cooperation between humans  and dinosaurs  [4].


**Source Citations:**


- Source  1: File_path: Data/Thundertooth Part  1 .docx (Lines:  1-10)


- Source  2: File_path: Data/Thundertooth Part  1 .docx (Lines:  11-20)


- Source  3: File_path: Data/Thundertooth Part  1 .docx (Lines:  21-30)


- Source  4: File_path: Data/Thundertooth Part  1 .docx (Lines:  31-40)

--------

2/4 Who was the main protagonist?

The text does NOT describe who was  the main protagonist therefore I cannot complete this query


**Note:** This text does NOT describe who was  the main protagonist therefore I am unable to complete 